In [33]:
import os
from pathlib import Path

path = Path('model.pkl')

!pip install -Uqq fastai

In [34]:
print(f"File exists: {path.exists()}")
print(f"File size: {path.stat().st_size if path.exists() else 'N/A'} bytes")
print(f"Full path: {path.absolute()}")
import zipfile

try:
    with zipfile.ZipFile(path, 'r') as zip_file:
        print("File is a valid ZIP archive")
except zipfile.BadZipFile:
    print("File is corrupted or not a valid ZIP archive")
except FileNotFoundError:
    print("File not found")


File exists: True
File size: 94592082 bytes
Full path: /Users/kalanjarvis-loewen/Desktop/Coding/learning/ml/LeanAI/core/model.pkl
File is a valid ZIP archive


In [35]:
from fastai.vision.all import *
model = load_learner("model.pkl")

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


In [36]:
def preprocess_img(pre_img):
    img = Image.open('images/0_image_1.jpg').convert('RGB')
    return transform(img).unsqueeze(0)

In [87]:
def predict_image_from_str(img):

    if img is None:
        return "Please upload an image first"
    
    try:
        print(f"Input type: {type(img)}")

        pil_img = PILImage.create(img)

        
        bf, _, preds = model.predict(pil_img)
        
        return float(preds[0])
        
    except Exception as e:
        return print(f"error: {e}")

pred = predict_image('images/1_image_1.jpg')               
print(pred)

Input type: <class 'str'>


9.547649383544922


In [89]:
import io

def predict_image_from_bytes(image_bytes: bytes):
    try:
        pil_img = PILImage.create(io.BytesIO(image_bytes))
        bf, _, preds = model.predict(pil_img)
        return float(preds[0])
    except Exception as e:
        return f"error: {e}"


In [90]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("# Body Fat Prediction")
    
    with gr.Row():
        image_input = gr.Image(label="Upload Image")
        output = gr.Number(label="Predicted Value")
    
    predict_btn = gr.Button("Predict")
    predict_btn.click(predict_image, inputs=image_input, outputs=output)

#demo.launch(share=True)



In [98]:
from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
import uvicorn
import threading

app = FastAPI()

# Allow requests from localhost:3000
app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:3000"],  # Add your frontend origin here
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post("/model/predict")
async def predict(request: Request):
    data = await request.json()
    img_data_uri = data['data'][0]  # get the base64 data URI string

    # Strip the prefix (e.g. 'data:image/png;base64,')
    _, encoded = img_data_uri.split(",", 1)

    # Decode to bytes
    img_bytes = base64.b64decode(encoded)

    # Use your existing function to get prediction
    pred = predict_image_from_bytes(img_bytes)

    return {"prediction": pred}

# Patch event loop for running in notebooks
nest_asyncio.apply()

# Run uvicorn in background thread
def run():
    uvicorn.run(app, host="0.0.0.0", port=8144)

threading.Thread(target=run).start()


INFO:     Started server process [55309]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8144 (Press CTRL+C to quit)


INFO:     127.0.0.1:54132 - "OPTIONS /model/predict HTTP/1.1" 200 OK


INFO:     127.0.0.1:54132 - "POST /model/predict HTTP/1.1" 200 OK


INFO:     127.0.0.1:54674 - "POST /model/predict HTTP/1.1" 200 OK


INFO:     127.0.0.1:54720 - "POST /model/predict HTTP/1.1" 200 OK


INFO:     127.0.0.1:55147 - "POST /model/predict HTTP/1.1" 200 OK


INFO:     127.0.0.1:55176 - "POST /model/predict HTTP/1.1" 200 OK


INFO:     127.0.0.1:55394 - "POST /model/predict HTTP/1.1" 200 OK
